Importando as bibliotecas necessárias

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

Extração dos urls das páginas com os vídeos e estatísticas das partidas

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://lolesports.com/pt-BR/vods/cblol-brazil/cblol_split_1_2024'
driver.get(url)

time.sleep(5)

video_url = []

base_xpath = '//*[@id="vodsRoot"]/div[2]/div[1]/div[{}]/div/div[2]/div[1]/div[{}]/*'

i, j = 2, 1 
tentativas = 0  

while True:
    try:
        button_xpath = base_xpath.format(i, j)
        button = driver.find_element(By.XPATH, button_xpath)
        video_url.append(button.get_attribute('href'))
        j += 1
        tentativas = 0
    except:
        if tentativas < 2:
            i += 1
            j = 1  
            tentativas += 1
        else:
            break

# Fecha o navegador
driver.quit()

Função para extrair o html de cada jogo, percorrendo a lista de urls e salvando o tempo da partida de acordo com a parte do vídeo (early, mid e late game):

In [ ]:
# Get the current working directory
current_directory = os.getcwd()

driver = webdriver.Chrome()
count = 1

def get_game_data(driver, count, game_time):
        #switch case to game_time ("early", "mid", "late")
        if game_time == "early":
            actions.send_keys(Keys.NUMPAD4).perform()
            time.sleep(8)
        elif game_time == "mid":
            actions.send_keys(Keys.NUMPAD6).perform()
            time.sleep(8)
        elif game_time == "late":
            actions.send_keys(Keys.NUMPAD9).perform()
            time.sleep(8)

        #get the htlm of the page
        html = driver.page_source

        file_name = f"{game_time}_lolesports_{count}.txt"
        # Obter o diretório do script atual
        script_directory = os.path.dirname(os.path.abspath(current_directory))
        
        # Construir o caminho completo para o arquivo 'lolesports.txt' dentro da pasta 'data/raw'
        file_path = os.path.join(script_directory, 'data', 'raw', file_name)
        # Garantir que o diretório 'data/raw' exista
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        # Salvar a saída no arquivo
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html)

for url in video_url:
    try:
        driver.get(url)
        
        actions = ActionChains(driver)

        print("Waiting for video to load")
        time.sleep(5)
        print("Video loaded")

        start_button = driver.find_element(By.XPATH, "//*[@class='attached']")
        start_button.click()

        time.sleep(5)

        get_game_data(driver, count, "early")
        get_game_data(driver, count, "mid")
        get_game_data(driver, count, "late")
        
        count += 1
    except Exception as e:
        print(e)
        
print("Closing driver")
driver.quit()

